<a href="https://colab.research.google.com/github/CiaraFarrellSETU/phd/blob/main/kmean_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
from google.colab import widgets
import os
import numpy as np
import geopandas as gpd
import rasterio
from shapely.geometry import Point
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from rasterio.windows import Window
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, cross_val_predict
from tqdm import tqdm
import time
import warnings
from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling
from sklearn.model_selection import StratifiedKFold, cross_val_predict
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import morphology
import joblib
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

zip_path = input("Please paste the path to your zip file from the sidebar: ")
destination = "/content/dataset"

if os.path.exists(zip_path):
    !unzip -q "{zip_path}" -d "{destination}"
    print(f"Successfully unzipped to {destination}")
else:
    print("Error: File path not found. Did you mount your drive?")
drive.mount('/content/drive')

Please paste the path to your zip file from the sidebar: /content/drive/MyDrive/scraghbog.zip
Successfully unzipped to /content/dataset
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

S_RGB_PATH = input("Summer RGB Path: ")
S_NDVI_PATH = input("Summer NDVI Path: ")
S_DSM = input("Summer DSM Path: ")
W_RGB_PATH = input("Winter RGB Path: ")
W_NDVI_PATH = input("Winter NDVI Path: ")
W_DSM = input("Winter DSM Path: ")
SHP_PATH = input("Shapefile (.shp) Path: ")

Summer RGB Path: /content/dataset/scraghbog/scraghbog_rgb_summer.tif
Summer NDVI Path: /content/dataset/scraghbog/Scraghbog_Summer_NDVI.tif
Summer DSM Path: /content/dataset/scraghbog/scraghbog_dem_summer.tif
Winter RGB Path: /content/dataset/scraghbog/Scraghbog_RGB_winter.tif
Winter NDVI Path: /content/dataset/scraghbog/Scraghbog_Winter_NDVI.tif
Winter DSM Path: /content/dataset/scraghbog/Scraghbog_Winter_DSM.tif
Shapefile (.shp) Path: /content/dataset/scraghbog/scraghBog_habiMap.shp


In [3]:
LABEL_COL = 'fid'
NAME_COL = 'HCH_MAPPED'

gdf = gpd.read_file(SHP_PATH)


KERNEL = 17
TARGET_RES = 0.5 # Meters
EPS = 1e-7


In [10]:
def resample_raster(path, target_res, master_params=None):
    with rasterio.open(path) as src:
        if master_params is None:
            transform, width, height = calculate_default_transform(
                src.crs, src.crs, src.width, src.height, *src.bounds, resolution=target_res)
            dst_crs = src.crs
        else:
            transform = master_params['transform']
            width = master_params['width']
            height = master_params['height']
            dst_crs = master_params['crs']

        data = np.zeros((src.count, height, width), dtype='float32')
        reproject(
            source=rasterio.band(src, list(range(1, src.count + 1))),
            destination=data,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear
        )
        return data, {'transform': transform, 'width': width, 'height': height, 'crs': dst_crs}

In [11]:
print("\n[1/4] Resampling and Aligning all layers to 0.5m...")
s_rgb_data, m_params = resample_raster(S_RGB_PATH, TARGET_RES)
s_nd_data, _  = resample_raster(S_NDVI_PATH, TARGET_RES, m_params)
s_dsm_data, _ = resample_raster(S_DSM,       TARGET_RES, m_params)
w_rgb_data, _ = resample_raster(W_RGB_PATH, TARGET_RES, m_params)
w_nd_data, _  = resample_raster(W_NDVI_PATH, TARGET_RES, m_params)
w_dsm_data, _ = resample_raster(W_DSM,       TARGET_RES, m_params)

layers = {'s_rgb': s_rgb_data, 's_ndvi': s_nd_data, 's_dsm': s_dsm_data,
          'w_rgb': w_rgb_data, 'w_ndvi': w_nd_data, 'w_dsm': w_dsm_data}

print("[2/4] Extracting every pixel inside polygons...")
gdf = gpd.read_file(SHP_PATH).to_crs(m_params['crs'])

fossitt_merger = {
    'GS4': 'Grassland', 'PF1' : 'fen & flush', 'PF3': 'bog',
    'WD4': 'Tree plantation', 'WN6': 'willow', 'WN7': 'bog woodland',
    'FS1': 'Swamp reeds', 'FS2': 'Swamp', 'FW2/WL2': 'water'
}
gdf['merged_name'] = gdf[NAME_COL].replace(fossitt_merger)
le = LabelEncoder()
gdf['class_encoded'] = le.fit_transform(gdf['merged_name'])

X_list, y_list = [], []

for _, row in gdf.iterrows():
    try:
        poly_pixels = {}
        for k, data in layers.items():
            # MemoryFile used for fast masking in RAM
            with rasterio.io.MemoryFile() as memfile:
                with memfile.open(driver='GTiff', height=m_params['height'], width=m_params['width'],
                                  count=data.shape[0], crs=m_params['crs'],
                                  transform=m_params['transform'], dtype='float32') as mem:
                    mem.write(data)
                    out_img, _ = mask(mem, [row['geometry']], crop=True)
                    poly_pixels[k] = out_img

        # Mask 2D: identifying valid pixels (not NoData)
        mask_2d = ~np.isnan(poly_pixels['s_ndvi'][0])
        idx = np.where(mask_2d)

        if len(idx[0]) > 0:
            # Extract raw values for every pixel
            s_r, s_g, s_b = poly_pixels['s_rgb'][:, idx[0], idx[1]]
            w_r, w_g, w_b = poly_pixels['w_rgb'][:, idx[0], idx[1]]
            s_nd = poly_pixels['s_ndvi'][0, idx[0], idx[1]]
            w_nd = poly_pixels['w_ndvi'][0, idx[0], idx[1]]
            s_h = poly_pixels['s_dsm'][0, idx[0], idx[1]]
            w_h = poly_pixels['w_dsm'][0, idx[0], idx[1]]

            # Feature Calculation (Pixel-wise)
            s_vari = (s_g - s_r) / (s_g + s_r - s_b + EPS)
            w_vari = (w_g - w_r) / (w_g + w_r - w_b + EPS)

            # Stack 12 basic features
            pixel_features = np.column_stack([
                s_r, s_g, s_b, w_r, w_g, w_b,
                s_nd, w_nd, s_nd - w_nd, # Delta NDVI
                s_vari, w_vari,
                s_h, w_h, s_h - w_h     # Delta Height
            ])

            X_list.append(pixel_features)
            y_list.append(np.full(len(pixel_features), row['class_encoded']))
    except: continue

X = np.nan_to_num(np.vstack(X_list))
y = np.concatenate(y_list)


[1/4] Resampling and Aligning all layers to 0.5m...
[2/4] Extracting every pixel inside polygons...


In [12]:
# --- PIXEL COUNT DIAGNOSTICS ---
print("\n" + "-"*30)
print("PIXELS EXTRACTED PER CLASS")
print("-"*30)

# Count the occurrences of each encoded class in y
unique, counts = np.unique(y, return_counts=True)

# Create a summary table
pixel_counts = pd.DataFrame({
    'Class Name': [le.classes_[i] for i in unique],
    'Pixel Count': counts
}).sort_values(by='Pixel Count', ascending=False)

print(pixel_counts.to_string(index=False))
print(f"\nTotal Pixels for Training: {X.shape[0]}")
print("-"*30 + "\n")


------------------------------
PIXELS EXTRACTED PER CLASS
------------------------------
     Class Name  Pixel Count
            bog      1144900
          Swamp       711713
         willow       388440
   bog woodland       376113
Tree plantation       229830
      Grassland       209341
    fen & flush       152208
          water        70210
    Swamp reeds        15882

Total Pixels for Training: 3298637
------------------------------



In [14]:
# --- 1. SUB-SAMPLING ---
MAX_PIXELS_PER_CLASS = 100000
X_balanced, y_balanced = [], []

print(f"Sub-sampling classes to a max of {MAX_PIXELS_PER_CLASS}...")
for class_idx in np.unique(y):
    indices = np.where(y == class_idx)[0]
    if len(indices) > MAX_PIXELS_PER_CLASS:
        indices = np.random.choice(indices, MAX_PIXELS_PER_CLASS, replace=False)
    X_balanced.append(X[indices])
    y_balanced.append(y[indices])

X = np.vstack(X_balanced)
y = np.concatenate(y_balanced)

# --- 2. CRITICAL CLEANING ---
# Replace Inf with large numbers and NaN with 0
X = np.nan_to_num(X, nan=0.0, posinf=1e6, neginf=-1e6)

# Clip to ensure no extreme outliers break the Scaler
X = np.clip(X, -1e6, 1e6)

print(f"Cleaned dataset size: {len(X)} pixels.")

# --- 3. SCALING & K-MEANS ---
from sklearn.cluster import MiniBatchKMeans

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Running MiniBatchKMeans...")
kmeans = MiniBatchKMeans(n_clusters=12, random_state=42, batch_size=2048, n_init=3)
clusters = kmeans.fit_predict(X_scaled)

X_combined = np.column_stack([X, clusters])

# --- 4. RANDOM FOREST ---
print("Training Random Forest...")
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    n_jobs=-1,
    random_state=42,
    class_weight='balanced'
)
rf.fit(X_combined, y)

print("\n" + "="*45)
print("FINAL PIXEL-LEVEL REPORT")
print("="*45)
y_pred = rf.predict(X_combined)
print(classification_report(y, y_pred, target_names=le.classes_))

Sub-sampling classes to a max of 100000...
Cleaned dataset size: 786092 pixels.
Running MiniBatchKMeans...
Training Random Forest...

FINAL PIXEL-LEVEL REPORT
                 precision    recall  f1-score   support

      Grassland       0.95      0.13      0.23    100000
          Swamp       0.88      0.12      0.21    100000
    Swamp reeds       0.59      0.64      0.61     15882
Tree plantation       1.00      0.43      0.60    100000
            bog       0.86      0.26      0.40    100000
   bog woodland       0.88      0.54      0.67    100000
    fen & flush       0.93      0.34      0.50    100000
          water       0.13      1.00      0.23     70210
         willow       0.86      0.29      0.44    100000

       accuracy                           0.37    786092
      macro avg       0.79      0.42      0.43    786092
   weighted avg       0.83      0.37      0.42    786092

